In [9]:
# convert json to csv

import json
import csv

# Open the JSON file and load the data into a list of dictionaries
with open("quotes.json", "r", encoding="utf-8") as f:
    data = json.load(f)["quotes"]


# Open the CSV file for writing and write the header row
with open("quotes.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["text", "author"])
    writer.writeheader()

    # Write each row to the CSV file
    for quote in data:
        writer.writerow(quote)


In [2]:
import requests
import json
import random
import pandas as pd

In [5]:
df = pd.read_csv("data/quotes.csv")

In [6]:
df.sample(10)

,text,author
1241,Retire into thyself. The rational principle wh...,Marcus Aurelius
1668,"Do, soul, do; abuse and contemn thyself; yet a...",Marcus Aurelius
1163,"Live out your life in truth and justice, toler...",Marcus Aurelius
1033,Most men ebb and flow in wretchedness between ...,Seneca
625,Peace you can claim for yourself without being...,Seneca
1543,He who has seen the present has seen everythin...,Marcus Aurelius
388,It is of course better to know useless things ...,Seneca
1761,Either teach them better if it be in thy power...,Marcus Aurelius
1001,"To know how many are jealous of you, count you...",Seneca
681,"If you wish to make Pythocles an old man, fill...",Seneca


In [7]:
df.shape

(1774, 2)

In [8]:
df.describe()

,text,author
count,1774,1774
unique,1726,4
top,"While we are postponing, life speeds by.",Seneca
freq,3,768


In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df["author"].value_counts()

author
Seneca             747
Marcus Aurelius    681
Epictetus          302
Cleanthes            1
Name: count, dtype: int64

In [11]:
df = df[df["author"] != "Cleanthes"]
# df["author"].value_counts()
df

,text,author
0,If anyone tells you that a certain person spea...,Epictetus
1,Wealth consists not in having great possession...,Epictetus
2,There is only one way to happiness and that is...,Epictetus
3,Man is not worried by real problems so much as...,Epictetus
4,First say to yourself what you would be; and t...,Epictetus
...,...,...
1768,It is royal to do good and to be abused.,Marcus Aurelius
1769,"Things?—I was once a fortunate man, but I lost...",Marcus Aurelius
1770,"Even chance is not divorced from nature, from ...",Marcus Aurelius
1771,"Be not either a man of many words, or busy abo...",Marcus Aurelius


In [12]:
df.to_csv('cleanData.csv', index=False)


In [13]:
randomRecord = df.sample()
text_value = randomRecord['text'].iloc[0]
author_value = randomRecord['author'].iloc[0]

In [14]:
text_value

'Restless people often pretend to be calm.'

In [15]:
author_value

'Seneca'

In [16]:
len(text_value)

41

In [17]:
def break_text(text):
    words = text.split()
    n = len(words)
    lines = []
    line = ""
    for i, word in enumerate(words):
        if len(line) + len(word) > 30:
            lines.append(line.strip())
            line = ""
        line += word + " "
        if i == n-1:
            lines.append(line.strip())
    return "\n".join(lines)

In [19]:
import math

def get_reading_time(text):
    # The average reading speed of an adult is 200 to 300 words per minute
    # We'll use 250 words per minute for our calculation
    words_per_minute = 100
    words = text.split()
    num_words = len(words)
    reading_time_minutes = num_words / words_per_minute
    reading_time_seconds = math.ceil(reading_time_minutes * 60)
    return reading_time_seconds

get_reading_time(text_value)

5

In [20]:
import os
import random

# Set the path to the folder containing the MP3 files
folder_path = "musicFinal/"

# Get a list of all MP3 files in the folder
mp3_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".mp3")]

# Select a random MP3 file from the list
audio_path = random.choice(mp3_files)

print(audio_path)


musicFinal/Haunted Forest - TrackTribe.mp3


In [21]:
# Set the file paths
if author_value == "Epictetus":
    period = "55 AD - 135 AD"
    image_path = "photos/epictetusFinal.jpeg"
elif author_value == "Seneca":
    period = "4 BC - 65 AD"
    image_path = "photos/senecaFinal.jpeg"
else:
    period = "121 AD - 180 AD"
    image_path = "photos/marcusFinal.jpeg"
    
print(image_path)

photos/senecaFinal.jpeg


In [22]:
from moviepy.editor import *
output_path = "output/output.mp4"

# Load the image file
image_clip = ImageClip(image_path).set_duration(get_reading_time(text_value)).set_fps(24)

# Lower the brightness of the image
image_clip = image_clip.fx(vfx.colorx, 0.7)

# Load the audio file and set its duration to match the video
audio_clip = AudioFileClip(audio_path).set_duration(image_clip.duration).audio_fadeout(1.0)

# Set the audio of the video
video_clip = image_clip.set_audio(audio_clip)

# Create the text clip and set its position and duration
fontsize = 130
quote_clip = TextClip(break_text(text_value), fontsize=fontsize, color='white', bg_color='transparent', align='center', stroke_color='black', stroke_width=4)
quote_clip = quote_clip.set_pos('center').set_duration(image_clip.duration)

# Add bold text clip below the first text clip
author_clip = TextClip(author_value, fontsize=fontsize+70, color='white', bg_color='transparent', font='Helvetica-BoldItalic', stroke_color='black', stroke_width=6)
author_clip = author_clip.set_pos(('center', 2400)).set_duration(image_clip.duration)

#Add period text clip below author text clip
period_clip = TextClip(period, fontsize=fontsize-50, color='white', bg_color='transparent', stroke_color='black', stroke_width=2)
period_clip = period_clip.set_pos(('center', 2650)).set_duration(image_clip.duration)

# Composite the text clip onto the video clip
video_clip = CompositeVideoClip([video_clip, quote_clip, author_clip, period_clip])

# Save the video to output folder
# video_clip.write_videofile(output_path, fps=24, codec="libx264")


In [23]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import moviepy.editor as mp
import io

def upload_to_youtube(video_clip, title, description, tags=None):
    scopes = ["https://www.googleapis.com/auth/youtube.upload"]

    # Authenticate using the client_secret.json file
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes)
    credentials = flow.run_local_server(port=0)
    youtube = googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

    # Convert video clip to bytes
    video_bytes = io.BytesIO()
    video_clip.write_videofile(video_bytes, fps=24, codec="libx264")
    video_bytes.seek(0)

    # Upload the video
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": tags if tags else [],
                "categoryId": "22"  # Entertainment category
            },
            "status": {
                "privacyStatus": "public"
            }
        },
        media_body=googleapiclient.http.MediaIoBaseUpload(video_bytes, mimetype='video/mp4', resumable=True)
    )

    response = request.execute()
    print(f"Video uploaded with video ID: {response['id']}")

# Assuming 'video_clip' is your MoviePy video clip object
title = "Stoic for life"
description = "Stoicism promotes emotional resilience by teaching us to focus on what we can control, and accept what we cannot, leading to inner peace and clarity. #Shorts"
tags = ["Shorts", "Stoicism"]

upload_to_youtube(video_clip, title, description, tags)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=693698111836-vadh7g9rnteeu2g0m8koel7qpmhe2462.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53982%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=FZAjDshHaojV2mkmbFOcdYv3nD5lBT&access_type=offline


TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

In [12]:
import azure.cognitiveservices.speech as speechsdk

# Set up the subscription and region keys
speech_key = '542e53c7056d449dbfa51b952c150f2a'
service_region = 'southeastasia'

# Set up the TTS client with desired voice
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.speech_synthesis_voice_name = 'en-US-JasonNeural'

synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

# Replace the variable with text
text_to_speak = "Observe always that everything is the result of change"
formatted_text = "<speak version='1.0' xml:lang='en-US'><voice name='en-US-JasonNeural'><s /><mstts:express-as style='hopeful'><prosody rate='-20.00%%' pitch='-10.00%%'>{}</prosody></mstts:express-as><s /></voice></speak>".format(text_value)

# Synthesize the text with SSML tags and the replaced variable
result = synthesizer.speak_ssml_async(formatted_text).get()

# Save the synthesized audio to a file
with open('output.wav', 'wb') as f:
    f.write(result.audio_data)

print('Speech synthesized and saved to output.wav')


Speech synthesized and saved to output.wav


In [122]:
import os

# get the current working directory
current_dir = os.getcwd()

# list all the files in the current directory
files = os.listdir(current_dir)

# print the files
for file in files:
    print(file)


.ipynb_checkpoints
audio.mp3
chosen_image.jpg
output.wav
quotes - Copy.csv
quotes.csv
quotes.json
Untitled.ipynb
video.mp4


In [114]:
import requests
import json
import random

access_key = "az1Om9jG8KXPKGXcYtYHIza8vBMFjqZ5CXEMyU72F-0"
search_query = author_value

# Set up the API endpoint and headers
endpoint = "https://api.unsplash.com/search/photos"
headers = {"Authorization": f"Client-ID {access_key}"}

# Make a GET request to the API endpoint with the search query
response = requests.get(endpoint, headers=headers, params={"query": search_query})

# Check the response status code
if response.status_code == 200:
    # Parse the response to get the list of portrait image URLs
    image_urls = [result["urls"]["regular"] for result in response.json()["results"] if result["height"] > result["width"]]
    
    if len(image_urls) > 0:
        # Choose a random image URL from the list
        chosen_image_url = random.choice(image_urls)
        
        # Download the image
        response = requests.get(chosen_image_url)
        with open("chosen_image.jpg", "wb") as f:
            f.write(response.content)
    else:
        print("No portrait images found.")
else:
    print(f"Unsplash API returned status code {response.status_code}")
    print(response.json())
